# Feed forward neural network to predict price per night

In [7]:
import json
import yaml
import torch
import torch.nn as nn

### Load best model

In [9]:
class FeedforwardNeuralNetModel(nn.Module):
    # Initialize the layers
    def __init__(self, input_dim, hidden_dim_array, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        self.hidden_dim_array = hidden_dim_array
        self.layers = nn.ModuleList()
        for i in range(len(hidden_dim_array)):
            self.layers.append(nn.Linear(input_dim, hidden_dim_array[i]))
            input_dim = hidden_dim_array[i]  # For the next layer
            self.layers.append(nn.ReLU())  # Activation function
        self.layers.append(nn.Linear(input_dim, output_dim))
    
    # Perform the computation
    def forward(self, x):
        output = x
        for layer in self.layers:
            output = layer(output)
        return output

model = torch.load('trains/2022-10-13_18:56:00/model.pt')
with open('trains/2022-10-13_18:56:00/hyperparameters.yml', 'r') as file:
    params = yaml.load(file, Loader=yaml.Loader)
params

{'hidden_dim_array': [6, 2],
 'learning_rate': 1e-05,
 'optimiser': torch.optim.sgd.SGD}